In [74]:
import torch
import random
import statistics
import numpy as np

import poselib

from tqdm import tqdm
from utils.rotation_utils import get_random_upward
from double_sol import SolverPipeline, DisplacementRefinerSolver
from SolverPipeline import P3PBindingWrapperPipeline

In [62]:
import sys
sys.executable

'/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8'

In [2]:
# config
from dataclasses import dataclass

@dataclass
class Config:
    max_depth: float = 10.
    img_width: int = 640
    img_height: int = 640
    focal_length: int = 3 * (img_width * 0.5) / np.tan(60.0 * np.pi / 180.0);
    min_depth: float = 1.
    max_depth: float = 1.1
    inliers_ratio: float = 1.
    outlier_dist: float = 30.
    
    # [TODO][IMPORTNAT]: not properly tested, be aware of using for
    # some experiments
    pixel_noise: float = 0.
    
conf = Config()
print(conf)

Config(max_depth=1.1, img_width=640, img_height=640, focal_length=554.2562584220409, min_depth=1.0, inliers_ratio=1.0, outlier_dist=30.0, pixel_noise=0.0)


In [3]:
ref = DisplacementRefinerSolver(verbose=False)            
p3p_solv_pipe = SolverPipeline()
p2p_solv_pipe = SolverPipeline(up2p=True)

In [49]:
def get_random_image_point(conf: Config):
    x = random.uniform(0, conf.img_width)
    y = random.uniform(0, conf.img_height)
    x = torch.tensor([x, y], dtype=torch.float64)    
    return x

def to_homogeneous(x):
    return torch.cat([x, torch.ones(1)])

def to_camera_coords(x: torch.tensor, conf: Config = conf):
    x = to_homogeneous(x)
    
    x[0] -= conf.img_width // 2
    x[1] -= conf.img_height // 2
    x[:2] /= conf.focal_length
    x /= x.norm()
    
    return x

def generate_correspondence(x: torch.tensor, conf: Config):
    x = to_camera_coords(x, conf)
    x *= random.uniform(conf.min_depth, conf.max_depth)
    
    assert x.shape == (3,)    
    return x

def transform_correspondence(X: torch.tensor, R: torch.tensor, t: torch.tensor):
    return R @ X + t

def generate_example(R, t, conf: Config = conf):
    x1, x2 = get_random_image_point(conf), get_random_image_point(conf)
    X1, X2 = generate_correspondence(x1.clone(), conf),\
             generate_correspondence(x2.clone(), conf)
    X1, X2 = transform_correspondence(X1, R, t), transform_correspondence(X2, R, t)
    
    # [TODO][IMPORTNAT]: not properly tested, be aware of using for
    # some experiments
    if conf.pixel_noise != 0:
        x1noise = np.random.normal(0, conf.pixel_noise, 2)
        x2noise = np.random.normal(0, conf.pixel_noise, 2)
        
        if torch.all(x1[0] + x1noise > 0) and torch.all(x1[1] + x1noise < conf.img_width):
            x1 += x1noise
            assert x1[0] > 0 and x1[1] < conf.img_width, f"{x1}"
            
        if torch.all(x2[0] + x2noise > 0) and torch.all(x2[1] + x2noise < conf.img_height):
            x2 += x2noise
            assert x2[0] > 0 and x2[1] < conf.img_height, f"{x2}"
        
        assert x1[0] > 0 and x1[1] < conf.img_width, f"{x1}"
        assert x2[0] > 0 and x2[1] < conf.img_height, f"{x2}"

    return x1, x2, X1, X2 

# x: [2, ]
def generate_outlier(x, conf):
    out = get_random_image_point(conf)
    
    while ((out - x).norm() < conf.outlier_dist):
        out = get_random_image_point(conf)
        
    return out

from typing import Tuple
def generate_examples(num_of_examples: int,
                      dev: Tuple[float, float] = (0., 0.), conf: Config = conf):
    num_of_examples = num_of_examples // 2
    
    num_inliers = num_of_examples * conf.inliers_ratio
    num_outliers = num_of_examples - num_inliers
    
    if num_of_examples == 0:
        num_of_examples, num_inliers, num_outliers = 1, 1, 0
    
    R, rand_angle = get_random_upward(*dev)
    t = torch.rand(3, )
        
    # [TODO] [IMPORTANT]: under such generation we cannot get model where one of the points is an inlier
    xs, Xs, inliers = [], [], []
    for i in range(num_of_examples):
        x1, x2, X1, X2 = generate_example(R, t)
        Xs.append((X1, X2))
        
        if i < num_inliers:
            xs.append((x1, x2))
            inliers.append(True)
        else:
            xs.append((generate_outlier(x1, conf), generate_outlier(x2, conf)))
            inliers.append(False)
            
    xs = np.concatenate([[p.numpy() for p in elm] for elm in xs])
    Xs = np.concatenate([[p.numpy() for p in elm] for elm in Xs])
    
            
    return xs, Xs, inliers, R, t, rand_angle

def compute_metric(Rgt, tgt, R, t):
    if R is None or t is None:
        return 1000000.0, 180.0
      
    rot_error = np.arccos((np.trace(np.matmul(R.T, Rgt)) - 1.0) / 2.0) * 180.0 / np.pi
    if np.isnan(rot_error):
        return 1000000.0, 180.0
    else:
        return np.linalg.norm(tgt - t), rot_error

In [59]:
ref = DisplacementRefinerSolver(verbose=False)            
p3p_solv_pipe = SolverPipeline()
p2p_solv_pipe = SolverPipeline(up2p=True)
p3pwrapper = P3PBindingWrapperPipeline(
        ransac_conf = {
            # 'max_reproj_error': args.ransac_thresh
            'min_iterations': 100,
            'max_iterations': 10000,
            'progressive_sampling': True,
            'max_prosac_iterations': 13
            },
            
            bundle_adj_conf = {
                'loss_scale' : 1.0,
            }                                              
    )

In [71]:
camera_dict = {
    "width": conf.img_width, 
    "height": conf.img_height, 
    "params": [conf.focal_length, conf.img_width // 2, conf.img_height // 2, 0]
}

orientation_errors_p3pr, pose_errors_p3pr = [], []
orientation_errors_p3p, pose_errors_p3p = [], []
orientation_errors_np, pose_errors_np = [], []
orientation_errors_p, pose_errors_p = [], []
    
seed = 13
for _ in tqdm(range(3)):
    xs, Xs, _, Rgt, tgt, rand_angle = generate_examples(10, (0, 0), conf)
    
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    # R, t = p3pwrapper(xs, Xs, camera_dict)
    # pose_error_p3pr, orient_error_p3pr = dataset.compute_metric(Rgt, tgt, R, t)
    # # print("rp3p[pe, oe]: ", pose_error_p3pr, orient_error_p3pr, Rotation.from_matrix(R).as_euler("XYZ", degrees=True), t)
    # orientation_errors_p3pr.append(orient_error_p3pr)
    # pose_errors_p3pr.append(pose_error_p3pr)

    np.random.seed(seed)
    torch.manual_seed(seed)
    R, t = p3p_solv_pipe(xs, Xs, camera_dict) 
    pose_error_p3p, orient_error_p3p = compute_metric(Rgt, tgt, R.numpy(), t.numpy())
    # print("p3p[pe, oe]: ", pose_error_p3p, orient_error_p3p, Rotation.from_matrix(R).as_euler("XYZ", degrees=True), t.numpy())
    orientation_errors_p3p.append(orient_error_p3p)
    pose_errors_p3p.append(pose_error_p3p)

    np.random.seed(seed)
    torch.manual_seed(seed)
    R, t = p2p_solv_pipe(xs, Xs, camera_dict) 
    pose_error_np, orient_error_np = compute_metric(Rgt, tgt, R.numpy(), t.numpy())
    # print("np[pe, oe]: ", pose_error_np, orient_error_np, Rotation.from_matrix(R).as_euler("XYZ", degrees=True), t.numpy())
    orientation_errors_np.append(orient_error_np)
    pose_errors_np.append(pose_error_np)

    np.random.seed(seed)
    torch.manual_seed(seed)
    R, t = ref(xs, Xs, camera_dict)
    pose_error_p, orient_error_p = compute_metric(Rgt, tgt, R, t)
    # print("p[pe, oe]: ", pose_error_p, orient_error_p, Rotation.from_matrix(R).as_euler("XYZ", degrees=True), t)
    orientation_errors_p.append(orient_error_p)
    pose_errors_p.append(pose_error_p)

100%|██████████| 3/3 [00:38<00:00, 12.80s/it]


In [72]:
def print_stats(pose_errors, orientation_errors):
    pos_errors = pose_errors
    orient_errors = orientation_errors
    print(" Couldn't localize " + str(orientation_errors.count(180.0)) + " out of " + str(len(orientation_errors)) + " images") 
    print(" Median position error: " +  str(round(statistics.median(pos_errors),3)) + ", median orientation errors: " + str(round(statistics.median(orient_errors),2)))

    med_pos = statistics.median(pos_errors)
    med_orient = statistics.median(orient_errors)
    counter = 0
    for i in range(0, len(pose_errors)):
        if pose_errors[i] <= med_pos and orientation_errors[i] <= med_orient:
            counter += 1
    print(" Percentage of poses within the median: " + str(100.0 * float(counter) / float(len(pose_errors))) + " % ")

In [76]:
print_stats(pose_errors_p3p, orientation_errors_p3p)
print_stats(pose_errors_np, orientation_errors_np)
print_stats(pose_errors_p, orientation_errors_p)

 Couldn't localize 0 out of 3 images
 Median position error: 0.955, median orientation errors: 101.05
 Percentage of poses within the median: 66.66666666666667 % 
 Couldn't localize 0 out of 3 images
 Median position error: 0.689, median orientation errors: 116.92
 Percentage of poses within the median: 33.333333333333336 % 
 Couldn't localize 0 out of 3 images
 Median position error: 2.077, median orientation errors: 137.64
 Percentage of poses within the median: 33.333333333333336 % 
